In [27]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import rioxarray as rio
from pathlib import Path
import pandas as pd
import zarr

In [28]:
#done
#!wget https://zenodo.org/record/8055879/files/seasfire_v0.3.zip

In [29]:
#!unzip seasfire_v0.3.zip
#!ls

In [30]:
dataset=xr.open_dataset(Path.home() / 'Documents/Wildfire Project/SeasFireCube_v3.zarr', engine='zarr')

In [31]:
dataset

<xarray.Dataset>
Dimensions:                         (latitude: 720, longitude: 1440, time: 966)
Coordinates:
  * latitude                        (latitude) float64 89.88 89.62 ... -89.88
  * longitude                       (longitude) float64 -179.9 -179.6 ... 179.9
  * time                            (time) datetime64[ns] 2001-01-01 ... 2021...
Data variables: (12/59)
    area                            (latitude, longitude) float32 ...
    biomes                          (latitude, longitude) float32 ...
    cams_co2fire                    (time, latitude, longitude) float32 ...
    cams_frpfire                    (time, latitude, longitude) float32 ...
    drought_code_max                (time, latitude, longitude) float32 ...
    drought_code_mean               (time, latitude, longitude) float32 ...
    ...                              ...
    t2m_max                         (time, latitude, longitude) float32 ...
    t2m_mean                        (time, latitude, longitude) float32 ...
    t2m_min                         (time, latitude, longitude) float32 ...
    tp                              (time, latitude, longitude) float32 ...
    vpd                             (time, latitude, longitude) float32 ...
    ws10                            (time, latitude, longitude) float32 ...
Attributes:
    crs:          EPSG:4326
    description:  The SeasFire Cube is a scientific datacube for seasonal fir...
    title:        SeasFire Cube: A Global Dataset for Seasonal Fire Modeling ...

In [32]:

# Clip the xarray dataset by getting min and max lat and long and then passing it through the rio built in functions to clip
def get_clipped_xarray_dataset(ds, min_long, min_lat, max_long, max_lat):
    # CRS code 
    dsc = ds.rio.write_crs(4326) # Need to specify this as Ilektra mentioned 
    
    # Clip the dataset
    ds_clipped = dsc.rio.clip_box(minx= min_long, miny= min_lat, maxx= max_long,maxy= max_lat)
    
    return ds_clipped 

#California
ds_clipped = get_clipped_xarray_dataset(dataset, -124,32,-114,42)

In [37]:
ds_clipped['fcci_ba']

<xarray.DataArray 'fcci_ba' (time: 966, latitude: 40, longitude: 40)>
[1545600 values with dtype=float32]
Coordinates:
  * latitude     (latitude) float64 41.88 41.62 41.38 ... 32.62 32.38 32.12
  * longitude    (longitude) float64 -123.9 -123.6 -123.4 ... -114.4 -114.1
    spatial_ref  int64 0
  * time         (time) datetime64[ns] 2001-01-01 2001-01-09 ... 2021-12-27
Attributes:
    creator_notes:    Masked ocean with lsm variable. Missing years filled wi...
    description:      The ESA Fire Disturbance Climate Change Initiative (CCI...
    downloaded_from:  https://catalogue.ceda.ac.uk/uuid/3628cb2fdba443588155e...
    long_name:        Burned Areas from Fire Climate Change Initiative (FCCI)
    missing_years:    2021
    provider:         ESA CCI
    units:            hectares (ha)

In [33]:
variables = [
    'tp',
    'rel_hum',
    'ws10',
    't2m_mean',
    't2m_min',
    't2m_max',
    'swvl1',
    'swvl2',
    'swvl3',
    'swvl4',
    'lsm',
    'drought_code_max',
    'drought_code_mean',
    'fwi_max',
    'fwi_mean',
    'lst_day',
    'lai',
    'ndvi',
    'pop_dens',
    'lccs_class_0',
    'lccs_class_1',
    'lccs_class_2',
    'lccs_class_3',
    'lccs_class_4',
    'lccs_class_5',
    'lccs_class_6',
    'lccs_class_7',
    'lccs_class_8',
    'fcci_ba_valid_mask']
final_ds = ds_clipped[variables]
final_ds.sizes

Frozen({'time': 966, 'latitude': 40, 'longitude': 40})

In [35]:
#final_ds.to_zarr(store = r'/home/aaron/Documents/Wildfire Project/california_dataset.zarr')

In [36]:
final_ds['fcci_ba_valid_mask']

<xarray.DataArray 'fcci_ba_valid_mask' (time: 966)>
[966 values with dtype=int8]
Coordinates:
    spatial_ref  int64 0
  * time         (time) datetime64[ns] 2001-01-01 2001-01-09 ... 2021-12-27
Attributes:
    description:  fcci_ba_valid_mask signifies the time period for which vari...